In [21]:
from InSituToolkit.imaging_database import write_experiment
from helpers import getPositions, getIDs
import imaging_db.database.db_operations as db_ops
import imaging_db.filestorage.s3_storage as s3_storage
import imaging_db.filestorage.local_storage as local_storage
import imaging_db.utils.db_utils as db_utils
import os, csv, pickle

# information taken from InSituToolkit notebooks, which happened to use the LungInsitu dataset
# we must pull all image id's that contain "SDG" and make an experiment for them.
# We assume that each file is a single round of imaging

spot_channels = ['Opal_570', 'Opal_620', 'Opal_690']
nuc_channel = ['DAPI']
db_credentials = '/Users/andrew.cote/Documents/db_credentials.json'
credentials_str = db_utils.get_connection_str(db_credentials)

In [25]:
'''

We want to save the experiment files in a specific format / directory naming convention, i.e.

in "/Users/andrew.cote/Documents/In-Situ_Transcriptomics/LungInSitu/experiments/" 

The list of experiments reads like

/TH134_E2_B1_assay2/roi1/<experiment_files>
/TH134_E2_B1_assay2/roi2/<experiment_files>

For ease of analysis we construct a dictionary to be saved with the format:

[dataset_id1, dataset_id2, ...]
{dataset_id: [savepath, assay#]}

'''

base_path = "/Users/andrew.cote/Documents/In-Situ_Transcriptomics/LungInSitu/experiments/"
dict_save_name = 'dict_of_experiments.obj'
list_save_name = 'list_of_experiments.obj'
dict_of_datasets = {}
list_of_datasets = []

csv_file = open('../Documentation/metadata_lung.csv')
csv_reader = csv.reader(csv_file, delimiter=',')
line_count = 0

for row in csv_reader:
    if line_count > 0:
        
        # extract relevant columns from the csv file
        dataset_id = row[0]
        roi = row[-1]
        full_name = row[2]
        
        # construct the directory name according to the convention specified:
        idx = full_name.find('assay')
        dir_name = full_name[0:idx+6]
        assayNo = full_name[idx+5]
        save_path = base_path + dir_name
        save_path_exp = save_path + '/roi' + roi + '/'
        file_name = save_path_exp[76:].replace('/', '-')
        
        if not os.path.exists(save_path):
            os.mkdir(save_path)
        
        if not os.path.exists(save_path_exp):
            os.mkdir(save_path_exp)
        
            # pull positions from database and write the experiment file
            positions = getPositions(db_credentials, dataset_id)
        
            write_experiment(db_credentials, save_path_exp, [dataset_id],
                     spot_channels=spot_channels, nuc_channels=nuc_channel,
                     positions=positions
                    )
            print("wr. exp @: " + save_path_exp)
        
        dict_of_datasets[dataset_id] = [save_path_exp, file_name, assayNo]
        list_of_datasets.append(dataset_id)
        
    line_count += 1



In [26]:
filehandler = open(dict_save_name, 'wb')
pickle.dump(dict_of_datasets, filehandler)
filehandler.close()

filehandler = open(list_save_name, 'wb')
pickle.dump(list_of_datasets, filehandler)
filehandler.close()

In [27]:
# there are three unique codebooks to be used, depending on which assay number

from starfish.types import Axes, Coordinates, Features
import pickle

codebook_1 = [
      {
          Features.CODEWORD: [
              {Axes.ROUND.value: 0, Axes.CH.value: 0, Features.CODE_VALUE: 1},
          ],
          Features.TARGET: "NFIX"
      },
      {
          Features.CODEWORD: [
              {Axes.ROUND.value: 0, Axes.CH.value: 1, Features.CODE_VALUE: 1},
          ],
          Features.TARGET: "SFTPD"
      },
      {
          Features.CODEWORD: [
              {Axes.ROUND.value: 0, Axes.CH.value: 2, Features.CODE_VALUE: 1},
          ],
          Features.TARGET: "DLC1"
      },
  ]

codebook_2 = [
      {
          Features.CODEWORD: [
              {Axes.ROUND.value: 0, Axes.CH.value: 0, Features.CODE_VALUE: 1},
          ],
          Features.TARGET: "IGFBP3"
      },
      {
          Features.CODEWORD: [
              {Axes.ROUND.value: 0, Axes.CH.value: 1, Features.CODE_VALUE: 1},
          ],
          Features.TARGET: "NKX2-1"
      },
      {
          Features.CODEWORD: [
              {Axes.ROUND.value: 0, Axes.CH.value: 2, Features.CODE_VALUE: 1},
          ],
          Features.TARGET: "SELENBP1"
      },
  ]

codebook_3 = [
      {
          Features.CODEWORD: [
              {Axes.ROUND.value: 0, Axes.CH.value: 0, Features.CODE_VALUE: 1},
          ],
          Features.TARGET: "SERPINE1"
      },
      {
          Features.CODEWORD: [
              {Axes.ROUND.value: 0, Axes.CH.value: 1, Features.CODE_VALUE: 1},
          ],
          Features.TARGET: "GJB3"
      },
      {
          Features.CODEWORD: [
              {Axes.ROUND.value: 0, Axes.CH.value: 2, Features.CODE_VALUE: 1},
          ],
          Features.TARGET: "GJB2"
      },
  ]

codebook = {1:codebook_1, 2:codebook_2, 3:codebook_3}

filehandler = open('codebook.obj', 'wb')
pickle.dump(codebook, filehandler)
filehandler.close()

In [7]:
codebook[2]

[{'codeword': [{'r': 0, 'c': 0, 'v': 1}], 'target': 'IGFBP3'},
 {'codeword': [{'r': 0, 'c': 1, 'v': 1}], 'target': 'NKX2-1'},
 {'codeword': [{'r': 0, 'c': 2, 'v': 1}], 'target': 'SELENBP1'}]